Here we implement the particle swarm optimisation algorithm.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
# the Particle object to be used in construction of the swarm
class Particle:
    def __init__(self, n_dim, initial = None):
        if initial is not None:
            self.position = initial[0]
            self.velocity = initial[1]
        else: 
            # if no initial position given, initialize with normal distribution
            self.position = torch.randn(n_dim)
            self.velocity = torch.randn(n_dim)
       
        self.best_position = self.position.clone()
        self.best_minimum = float('inf')   

class ParticleSwarm:
    def __init__(self, n_dim = 2, n_particles = 100, alpha = .95, beta = .1, gamma = .1):
        self.n_dim = n_dim # number of dimensions of the space
        self.n_particles = n_particles # number of particles in the swarm
        self.alpha = alpha # hyperparameters alpha, beta,, gamma
        self.beta = beta
        self.gamma = gamma
        self.global_best_position = None # the best initial position
        self.global_best_minimum = float('inf') # the best minimum encountered so far
        self.swarm = [Particle(n_dim) for _ in range(n_particles)]

    def optimise(self, f, n_iterations = 10, visual = False):
        # get the information on the state of the swarm
        positions = torch.stack([particle.position for particle in self.swarm])
        velocities = torch.stack([particle.velocity for particle in self.swarm])
        best_positions = torch.stack([particle.best_position for particle in self.swarm])
        best_minima = torch.tensor([particle.best_minimum for particle in self.swarm])
        
        if self.global_best_position is None:
            _, indices = torch.min(f(best_positions), dim=0)
            self.global_best_position = best_positions[indices]
        
        if visual:
            fig, ax = plt.subplots()
        
        # run the optimisation algorithm for n_iterations steps 
        for k in range(n_iterations):
            # update velocities
            a = self.alpha * velocities
            b = self.beta * torch.rand(self.n_particles, self.n_dim) * (best_positions - positions)
            c = self.gamma * torch.rand(self.n_particles, self.n_dim) * (self.global_best_position - positions)
            velocities = a + b + c
            
            # update positions
            positions += velocities
            
            # evaluate function values for new positions
            new_minima = f(positions)
            
            improved_indices = new_minima < best_minima
            best_positions[improved_indices] = positions[improved_indices]
            best_minima[improved_indices] = new_minima[improved_indices]
            
            # update the best particle position
            _, indices = torch.min(new_minima, dim=0)
            self.global_best_position = positions[indices]
            
            if visual:
                # plot particle positions
                ax.clear()
                ax.scatter(positions[:, 0], positions[:, 1], color = 'blue', label = 'Particles')
                ax.scatter(self.global_best_position[0], self.global_best_position[1], color = 'red', label = 'Global Best Minimum')
                ax.set_xlabel('X')
                ax.set_ylabel('Y')
                ax.set_title(f'Particle Swarm Optimization, step: {k+1}')
                ax.legend()
                plt.pause(.0001)
            
        # update the state of the swarm
        for i, particle in enumerate(self.swarm):
            particle.position = positions[i]
            particle.velocity = velocities[i]
            particle.best_position = best_positions[i]
            particle.best_minimum = best_minima[i].item()


In [3]:
# 1-d Rosenbrock function, minimum at (1,1)
f = lambda x: (1 - x[:, 0]) ** 2 + 100 * (x[:, 1] - x[:, 0] ** 2) ** 2

ps = ParticleSwarm(n_particles = 50)
ps.optimise(f, 500)
ps.global_best_position

tensor([1.0000, 1.0000])